In [69]:
from datasets import load_dataset
import spacy # Better than nltk
from spacy.lang.fr.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora
from gensim import models
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel
#import matplotlib.pyplot as plt

In [2]:
dataset = load_dataset('mlsum', 'fr')

Reusing dataset mlsum (C:\Users\maxim\.cache\huggingface\datasets\mlsum\fr\1.0.0\77f23eb185781f439927ac2569ab1da1083195d8b2dab2b2f6bbe52feb600688)


  0%|          | 0/3 [00:00<?, ?it/s]

In [78]:
article = dataset['train']['text'][4]
summary = dataset['train']['summary'][4]
print(article, summary, sep='\n')

Cinq personnes sont mortes, et treize autres ont été blessées à Nîmes, dans le Gard, dans un incendie qui s'est déclenché vendredi 1er janvier au petit matin. Le feu, dont on ignore l'origine pour l'instant, a pris au sixième et dernier étage d'un immeuble. "A l'arrivée des pompiers, trois personnes étaient décédées dans un appartement, et deux autres dans un appartement voisin par intoxication", a expliqué, sur i-Télé, le directeur de cabinet du préfet du Gard. On dénombre également "treize blessés, dont trois graves. Une personne dans le coma a été transférée à Marseille", a-t-il ajouté. Les secours ont été prévenus vers 5 heures du matin, mais "l'incendie avait déjà bien démarré", a-t-il expliqué. France Info précise que les victimes sont trois adultes et deux enfants. "L'origine de l'incendie est indéterminée mais a priori accidentelle", a déclaré le procureur adjoint de la République de Nîmes, cité par Europe 1.
Cinq personnes sont mortes, et treize autres ont été blessées à Nîmes

In [91]:
nlp = spacy.load("fr_core_news_sm")
doc = nlp(article)

sentences = []
cur_sentence = []
for sent in doc.sents:
    for token in sent:
        if not token.text.lower() in STOP_WORDS and not token.is_punct:
            # print(token.text, token.lemma_)
            cur_sentence.append(token.lemma_)
    sentences.append(cur_sentence)
    cur_sentence = []

for s in sentences:
    print(s)
    print()

['personne', 'mourir', 'être', 'blesser', 'Nîmes', 'Gard', 'incendie', 'déclencher', 'vendredi', 'premier', 'janvier', 'petit', 'matin']

['feu', 'ignore', 'origine', 'instant', 'prendre', 'dernier', 'étage', 'immeuble']

['arrivée', 'pompier', 'personne', 'décéder', 'appartement', 'appartement', 'voisin', 'intoxication', 'expliquer', 'Télé', 'directeur', 'cabinet', 'préfet', 'Gard']

['dénombre', 'blessé', 'grave']

['coma', 'être', 'transférer', 'Marseille', '-t', 'il', 'ajouté']

['secours', 'être', 'prévenir', '5', 'heure', 'matin', 'incendie', 'bien', 'démarrer', '-t', 'il', 'expliquer']

['France', 'Info', 'précise', 'victime', 'adulte', 'enfant']

['origine', 'incendie', 'indéterminer', 'priori', 'accidentel', 'déclarer', 'procureur', 'adjoint', 'république', 'nîme', 'citer', 'Europe', '1']



In [94]:
dictionary = corpora.Dictionary(sentences)
print(dictionary)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in sentences]
tfidf = models.TfidfModel(doc_term_matrix)
sentences_tfidf = tfidf[doc_term_matrix]
for sent in sentences_tfidf:
    print(sent)

Dictionary(64 unique tokens: ['Gard', 'Nîmes', 'blesser', 'déclencher', 'incendie']...)
[(0, 0.2131950688824599), (1, 0.31979260332368986), (2, 0.31979260332368986), (3, 0.31979260332368986), (4, 0.15083950856500844), (5, 0.31979260332368986), (6, 0.2131950688824599), (7, 0.31979260332368986), (8, 0.2131950688824599), (9, 0.31979260332368986), (10, 0.31979260332368986), (11, 0.31979260332368986), (12, 0.15083950856500844)]
[(13, 0.36650833306891567), (14, 0.36650833306891567), (15, 0.36650833306891567), (16, 0.36650833306891567), (17, 0.36650833306891567), (18, 0.24433888871261045), (19, 0.36650833306891567), (20, 0.36650833306891567)]
[(0, 0.17609018126512477), (8, 0.17609018126512477), (21, 0.26413527189768715), (22, 0.5282705437953743), (23, 0.26413527189768715), (24, 0.26413527189768715), (25, 0.26413527189768715), (26, 0.26413527189768715), (27, 0.17609018126512477), (28, 0.26413527189768715), (29, 0.26413527189768715), (30, 0.26413527189768715), (31, 0.26413527189768715)]
[(32, 0

In [96]:
coherence_values = []
model_list = []
for num_topics in range(2, 10):
    model = LsiModel(sentences_tfidf, num_topics=num_topics, id2word=dictionary)
    model_list.append(model)
    coherencemodel = CoherenceModel(model=model, texts=sentences, dictionary=dictionary)
    coherence_values.append(coherencemodel.get_coherence())

print(coherence_values)
max_coherence = coherence_values.index(max(coherence_values))
num_topics = 2 + max_coherence
model = model_list[max_coherence]
print(num_topics)


[0.37091476056602574, 0.36943770731509135, 0.3262268750177908, 0.34114646686964106, 0.34810325397783504, 0.3218931196738978, 0.3186808531865763, 0.30191561138347733]
2


In [97]:
print(model.projection.s)
print(model.print_topics(num_topics=num_topics))
print()
corpus_lsi = model[doc_term_matrix]
for sent in zip(corpus_lsi, doc.sents):
    print(sent)

[1.10966604 1.02402477]
[(0, '0.281*"-t" + 0.281*"il" + 0.260*"être" + 0.222*"matin" + 0.217*"coma" + 0.217*"transférer" + 0.217*"ajouté" + 0.217*"Marseille" + 0.205*"démarrer" + 0.205*"bien"'), (1, '-0.281*"origine" + -0.232*"immeuble" + -0.232*"feu" + -0.232*"ignore" + -0.232*"dernier" + -0.232*"étage" + -0.232*"prendre" + -0.232*"instant" + -0.189*"priori" + -0.189*"procureur"')]

([(0, 1.9391953183330828), (1, -0.6375553403982903)], Cinq personnes sont mortes, et treize autres ont été blessées à Nîmes, dans le Gard, dans un incendie qui s'est déclenché vendredi 1er janvier au petit matin.)
([(0, 0.07596665115149759), (1, -1.90426511794746)], Le feu, dont on ignore l'origine pour l'instant, a pris au sixième et dernier étage d'un immeuble.)
([(0, 1.238410950957065), (1, -0.7474116198399976)], "A l'arrivée des pompiers, trois personnes étaient décédées dans un appartement, et deux autres dans un appartement voisin par intoxication", a expliqué, sur i-Télé, le directeur de cabinet du 